In [ ]:
import tensorflow as tf
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng()

# Para interpolar
import scipy.interpolate
from scipy.interpolate import griddata

# Para hacer animaciones
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
# Se toma num^2 datos aleatorios para el entrenamiento
def gen_traindata(num):
    data = np.load("dataset/EcOnda1D_F_BCN.npz")
    t, x, exact = data["t"], data["x"], data["sol"].T
    lista_ind_t=rng.choice(len(t), num, replace=False)
    lista_ind_x=rng.choice(len(x), num, replace=False)
    xx, tt = np.meshgrid(x[lista_ind_x], t[lista_ind_t])
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    u = exact[lista_ind_t][:,lista_ind_x].flatten()[:, None]
    return X, u

# Medio homogeneo de velocidad constante
v=1
# Se define la ecuación diferencial que se pueda leer en tensorflow
def pde(x, y):
    u, f = y[:, 0:1], y[:, 1:2]
    du_xx = dde.grad.hessian(u, x, i=0, j=0)
    du_tt = dde.grad.hessian(u, x, i=1, j=1)
    return du_tt - v**2 * du_xx - f

def fun_bc(x):
    return 0

def fun_init(x):
    xc=0.2
    sigma=0.06
    return np.exp(-((x[:,0:1]-xc)/sigma)**2)

# Se define el dominio espacio-temporal
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Condiciones de frontera de Neumann iguales a cero
bc_u = dde.icbc.NeumannBC(geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=0)

# Condiciones iniciales de posición y velocidad
ic1 = dde.icbc.IC(geomtime, fun_init, lambda _, on_initial: on_initial, component=0)

def boundary_initial(x, _):
    return  np.isclose(x[-1], 0 )

ic2 = dde.NeumannBC(geomtime, fun_bc, boundary_initial, component=0)

# Datos de entrenamiento
observe_x, observe_u = gen_traindata(10)
observe_y1 = dde.icbc.PointSetBC(observe_x, observe_u, component=0)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_u, ic1, ic2, observe_y1],
    num_domain=360,
    num_boundary=360,
    num_initial=360,
    anchors=observe_x,
    num_test=100,
)

# Se define la arquitectura de la red neuronal FNN
layer_size = [2] + [20]*4 + [2]
activation="tanh"
initializer = "Glorot uniform"
optimizer = "adam"
net=dde.maps.FNN(layer_size,activation,initializer)

# Se define el modelo y se compila
model=dde.Model(data, net)
model.compile(optimizer, lr=0.001)
losshistory, train_state = model.train(iterations=40000)

# Se cambia el metodo de optimización y se sigue entrenando
model.compile("L-BFGS")
losshistory, train_state = model.train()
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [ ]:
data3 = np.load("dataset/EcOnda1D_F_BCN.npz")
t, x, exact = data3["t"], data3["x"], data3["sol"].T
x1 = x
t1 = t
xx1, tt1 = np.meshgrid(x1, t1)
X1 = np.vstack((np.ravel(xx1), np.ravel(tt1))).T
pred1SC = model.predict(X1)
u_pred1SC, f_pred1SC = pred1SC[:, 0:1], pred1SC[:, 1:2]

X_star1 = np.hstack((xx1.flatten()[:,None], tt1.flatten()[:,None]))
U_pred1SC = griddata(X_star1, u_pred1SC.flatten(), (xx1, tt1), method='cubic')
F_pred1SC = griddata(X_star1, f_pred1SC.flatten(), (xx1, tt1), method='cubic')

snaps_num = 5      # without first and last

snaps = [1]        #first slice

for k in range(1,snaps_num):
    snaps += [int(k*(len(t1)/snaps_num))]

snaps += [int(len(t1)-1)] #last slice

print("snapshots at:", snaps)

for snap in snaps:
    exa = plt.plot(x1,exact[snap-1,:],'r-', label = 'FD')
    pred = plt.plot(x1,U_pred1SC[snap,:],'b--', label = 'PINN')
    plt.title('$t = %1.3f s$ '%(t[snap]), fontsize = 15)
    plt.xlabel('x', fontsize = 14)
    plt.ylabel('u(x,t)', fontsize = 14)
    plt.legend(prop={'size':12})
    plt.show()

In [ ]:
# Visualización de los resultados

snaps = [1]        #primer figura

for k in range(1,snaps_num):
    snaps += [int(k*(len(t1)/snaps_num))]

snaps += [int(len(t1)-1)] #ultima figura

print("snapshots at:", snaps)

ftrue = lambda x, t: np.exp(-3*t)*np.sin(x)

for snap in snaps:
    exa = plt.plot(x1,ftrue(x1,t1[snap-1,:]),'r-', label = 'FD')
    pred = plt.plot(x1,F_pred1SC[snap,:],'b--', label = 'PINN')
    plt.title('$t = %1.3f s$ '%(t[snap]), fontsize = 15)
    plt.xlabel('x', fontsize = 10)
    plt.ylabel('f(x,t)', fontsize = 10)
    plt.legend(prop={'size':12})
    plt.show()

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(111)

def animate(i):
    ax.clear()
    ax.plot(x1,exact[i-1,:],'r-', label = 'FD')
    ax.plot(x1,U_pred1SC[i,:],'b--', label = 'PINN')
    ax.set_title('$t = %1.3f s$ '%(t[i]), fontsize = 15)
    ax.set_xlabel('x', fontsize = 10)
    ax.set_ylabel('u(x,t)', fontsize = 10)
    
anim = FuncAnimation(fig, animate, range(1,len(t1)-1), interval=100)
plt.show()
anim.save('Onda 1D DF No Homogeneo Caso 2.gif')